In [29]:
# Data Processor
#
# Desired output
# - Full train dataset - for training model for Kaggle submission
# - Full test dataset - for Kaggle submission
# - Internal train dataset - 80% of full train dataset for internal banchmarking
# - Internal test dataset - 20% of full train dataset for internal benchmarking
# - All tabular data in one file

import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import shutil


print("Processing data\n")

# Load data
print("Load data")
train_df = pd.read_pickle("processed_data/top500-train/top500-train.pkl")

Processing data

Load data


In [30]:
train_df.shape

(87396, 1488)

In [31]:
internal_train_df = train_df.sample(frac = 0.8, random_state = 113)
internal_test_df = train_df.drop(internal_train_df.index)

In [32]:
print(f"{internal_train_df.shape=}")
print(f"{internal_test_df.shape=}")

internal_train_df.shape=(69917, 1488)
internal_test_df.shape=(17479, 1488)


In [33]:
print("Export internal training data")
print(" - Make directory")
if not os.path.exists("processed_data/internal"):
    os.makedirs("processed_data/internal")
if not os.path.exists("processed_data/internal/train_images"):
    os.makedirs("processed_data/internal/train_images")

Export internal training data
 - Make directory


In [34]:
print(" - Export tabular data")
internal_train_df.to_pickle("processed_data/internal/internal_train.pkl")

 - Export tabular data


In [35]:
internal_train_df.head()

,surveyId,lon,lat,year,geoUncertaintyInM,areaInM2,Bio1,Bio2,Bio3,Bio4,...,speciesId_10991,speciesId_10998,speciesId_11005,speciesId_11054,speciesId_11078,speciesId_11120,speciesId_11140,speciesId_11157,speciesId_11193,speciesId_11195
27540,1233260,10.026100,55.231480,2018,10.0,79.0,2819,44,2,5649,...,False,False,False,False,False,False,False,False,False,False
14388,645374,11.809192,46.086903,2018,10.0,100.0,2760,91,3,6798,...,False,False,False,False,False,False,False,False,False,False
56427,2539031,14.889000,55.127910,2017,10.0,79.0,2811,19,1,5513,...,False,False,False,False,False,False,False,False,False,False
31056,1388787,9.775561,51.447618,2017,5.0,25.0,2816,70,2,6054,...,False,True,False,False,False,False,False,False,False,False
47984,2158677,4.997110,48.619160,2019,1.0,100.0,2840,73,3,5967,...,False,False,True,False,False,False,False,False,False,False


In [36]:
print(" - Copying over images")
for survey_id in tqdm(internal_train_df.surveyId):
    shutil.copy(f"processed_data/top500-train/top500-train_images/{survey_id}.png", f"processed_data/internal/train_images/{survey_id}.png")

 - Copying over images


100%|██████████| 69917/69917 [00:48<00:00, 1446.90it/s]


In [42]:
print("Make test data")
print(" - Make directory")
if not os.path.exists("processed_data/internal/test_images"):
    os.makedirs("processed_data/internal/test_images")

Make test data
 - Make directory


In [61]:
print("Split data")
internal_test_X_df = internal_test_df.loc[:,~internal_test_df.columns.str.startswith('speciesId_')]
species_columns = ['surveyId'] + list(internal_test_df.columns[internal_test_df.columns.str.startswith('speciesId_')])
internal_test_y_df = internal_test_df.loc[:,species_columns]

In [63]:
print("Save data")
print(" - test data")
internal_test_X_df.to_pickle("processed_data/internal/internal_test_X.pkl")

Save data
 - test data


In [64]:
print(" - Copying over images")
for survey_id in tqdm(internal_test_df.surveyId):
    shutil.copy(f"processed_data/top500-train/top500-train_images/{survey_id}.png", f"processed_data/internal/test_images/{survey_id}.png")

 - Copying over images


100%|██████████| 17479/17479 [00:09<00:00, 1900.57it/s]


In [74]:
species = [column[10:] for column in species_columns[1:]]

In [81]:
with open("processed_data/internal/submission.csv", "w") as f:
    f.write("surveyId,speciesId\n")
    for _, row in internal_test_y_df.iterrows():
        f.write(str(row[0]) + "," + " ".join([species[i] for i in range(1, len(row)) if row[i] == 1]) + "\n")

/tmp/ipykernel_27405/877506001.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f.write(str(row[0]) + "," + " ".join([species[i] for i in range(1, len(row)) if row[i] == 1]) + "\n")
